In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

     |████████████████████████████████| 6.4 MB 7.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 2.2 MB/s 


In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(11,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-10][3],
                df[each-9][3],
                df[each-8][3],
                df[each-7][3],
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["sugg"] = last_column
  return scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df      

In [ ]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")
from google.colab import drive
drive.mount("/content/gdrive")
symbols = get_crypto_syms()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
data = process(dfs)
data.to_csv("1d_10pastcandle_newversion.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data

Mounted at /content/gdrive

1 Failed download:
- XNO-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symb

,0,1,2,3,4,5,6,7,8,9,sugg
0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0


In [7]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.add(Dense(5000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(4096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(350, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5000)              55000     
                                                                 
 dense_1 (Dense)             (None, 4096)              20484096  
                                                                 
 dense_2 (Dense)             (None, 2048)              8390656   
                                                                 
 dense_3 (Dense)             (None, 750)               1536750   
                                                                 
 dense_4 (Dense)             (None, 350)               262850    
                                                                 
 dense_5 (Dense)             (None, 1)                 351       
                                                                 
Total params: 30,729,703
Trainable params: 30,729,703


In [ ]:
model.fit(xTrain,yTrain,epochs=200,batch_size=20000,validation_data=(xTest,yTest))
model.evaluate(xTest,yTest)
model.save(f"{model.evaluate(xTest,yTest)}.h5")

Epoch 1/200
80/80 [==============================] - 82s 984ms/step - loss: 0.6905 - accuracy: 0.5295 - val_loss: 0.6883 - val_accuracy: 0.5368
Epoch 2/200
80/80 [==============================] - 80s 1s/step - loss: 0.6850 - accuracy: 0.5475 - val_loss: 0.6838 - val_accuracy: 0.5513
Epoch 3/200
80/80 [==============================] - 81s 1s/step - loss: 0.6796 - accuracy: 0.5614 - val_loss: 0.6767 - val_accuracy: 0.5677
Epoch 4/200
80/80 [==============================] - 79s 994ms/step - loss: 0.6710 - accuracy: 0.5772 - val_loss: 0.6666 - val_accuracy: 0.5837
Epoch 5/200
80/80 [==============================] - 79s 983ms/step - loss: 0.6627 - accuracy: 0.5895 - val_loss: 0.6580 - val_accuracy: 0.5952
Epoch 6/200
80/80 [==============================] - 78s 976ms/step - loss: 0.6511 - accuracy: 0.6055 - val_loss: 0.6547 - val_accuracy: 0.5999
Epoch 7/200
80/80 [==============================] - 77s 969ms/step - loss: 0.6391 - accuracy: 0.6209 - val_loss: 0.6436 - val_accuracy: 0.616

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

In [ ]:
import os
files = os.listdir("/content/gdrive/MyDrive/Colab Files/")

data = pd.DataFrame()
for file in files:
  address = f"/content/gdrive/MyDrive/Colab Files/{file}"
  df = pd.read_csv(address)
  data = pd.concat([data,df])

In [ ]:
symbols = read_syms_from_txt()
len(symbols)
pieces = 15
new_arrays = np.array_split(symbols, pieces)
for i in new_arrays:
  dfs = []
  for x in i:
        data = yf.download(x,period="MAX",interval="1d",progress=False)
        if data.empty :
             print("Passing...")
        else:
               dfs.append(data)
  ndfs = dfs
  dfs = []
  for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
  data = process(dfs)
  data.to_csv(f"{x}.csv")         

In [4]:
address = "/content/gdrive/MyDrive/1d_10pastcandle_newversion.csv"
from google.colab import drive
drive.mount("/content/gdrive/")
data = pd.read_csv(address)


Mounted at /content/gdrive/


NameError: ignored

In [5]:
data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,sugg
0,0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1761617,1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0


In [6]:
data = data.drop(["Unnamed: 0"],axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(1585459, 10) (1585459,)
(176163, 10) (176163,)


,0,1,2,3,4,5,6,7,8,9,sugg
0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0
